In [397]:
import pandas as pd
import numpy as np
file = pd.read_csv('utah_sales_20241203_201859.csv')
df = pd.DataFrame(file)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.svm import SVC 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix



C:\Users\jpdav\AppData\Local\Temp\ipykernel_137824\2581941509.py:3: DtypeWarning: Columns (11,12,13,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  file = pd.read_csv('utah_sales_20241203_201859.csv')


In [398]:
df.head()

,id,formattedAddress,addressLine1,addressLine2,city,state,zipCode,county,latitude,longitude,...,squareFootage,lotSize,yearBuilt,status,price,listedDate,removedDate,createdDate,lastSeenDate,daysOnMarket
0,"231-S-Virginia-St,-Kaysville,-UT-84037","231 S Virginia St, Kaysville, UT 84037",231 S Virginia St,NaN,Kaysville,UT,84037,Davis,41.031291,-111.934739,...,862.0,4792.0,1941.0,Inactive,360000,2024-01-05T00:00:00.000Z,2024-04-06T00:00:00.000Z,2023-12-08T00:00:00.000Z,2024-04-05T00:00:00.000Z,92.0
1,"3157-Monroe-Blvd,-Ogden,-UT-84403","3157 Monroe Blvd, Ogden, UT 84403",3157 Monroe Blvd,NaN,Ogden,UT,84403,Weber,41.206655,-111.959974,...,1750.0,4792.0,1941.0,Inactive,314999,2024-01-05T00:00:00.000Z,2024-04-06T00:00:00.000Z,2024-01-06T00:00:00.000Z,2024-04-05T00:00:00.000Z,92.0
2,"1988-Stone-Creek-Rd,-Unit-19,-Park-City,-UT-84098","1988 Stone Creek Rd, Unit 19, Park City, UT 84098",1988 Stone Creek Rd,Unit 19,Park City,UT,84098,Summit,40.683172,-111.549270,...,2082.0,NaN,2025.0,Inactive,2790000,2024-01-10T00:00:00.000Z,2024-04-06T00:00:00.000Z,2022-07-27T14:22:11.749Z,2024-04-05T00:00:00.000Z,87.0
3,"1990-Stone-Creek-Rd,-Unit-20,-Park-City,-UT-84098","1990 Stone Creek Rd, Unit 20, Park City, UT 84098",1990 Stone Creek Rd,Unit 20,Park City,UT,84098,Summit,40.683081,-111.549202,...,2533.0,NaN,2025.0,Inactive,3190000,2024-01-10T00:00:00.000Z,2024-04-06T00:00:00.000Z,2022-07-27T14:24:30.407Z,2024-04-05T00:00:00.000Z,87.0
4,"4398-N-2525-W,-Cedar-City,-UT-84721","4398 N 2525 W, Cedar City, UT 84721",4398 N 2525 W,NaN,Cedar City,UT,84721,Iron,37.757465,-113.097258,...,3000.0,21780.0,2021.0,Inactive,540000,2024-01-12T00:00:00.000Z,2024-04-06T00:00:00.000Z,2023-12-31T00:00:00.000Z,2024-04-05T00:00:00.000Z,85.0


In [399]:
df.keys()

Index(['id', 'formattedAddress', 'addressLine1', 'addressLine2', 'city',
       'state', 'zipCode', 'county', 'latitude', 'longitude', 'propertyType',
       'bedrooms', 'bathrooms', 'squareFootage', 'lotSize', 'yearBuilt',
       'status', 'price', 'listedDate', 'removedDate', 'createdDate',
       'lastSeenDate', 'daysOnMarket'],
      dtype='object')

In [400]:
df['propertyType'].unique()

array(['Single Family', 'Townhouse', 'Condo', 'Land', 'Manufactured',
       'Multi-Family', 'Apartment', nan, '2.0', '3.0', '5.0', '7.0',
       '6.0', '4.0', '0.0', '10.0', '1.0', '8.0', '12.0'], dtype=object)

In [401]:
dropped = df.drop(columns=['id', 'formattedAddress', 'addressLine1', 'addressLine2', 'city',
       'state', 'latitude', 'longitude', 'listedDate', 'removedDate', 'createdDate', 'lastSeenDate', 'status', 'county'])

In [402]:
# if dropped['propertyType'] == ['Apartment','Manufactured',
#        'Multi-Family','Land', 'nan', '2.0', '3.0', '5.0', '7.0',
#        '6.0', '4.0', '0.0', '10.0', '1.0', '8.0', '12.0']:
    

In [403]:
my_conditions = ['Apartment','Manufactured',
       'Multi-Family','Land', 'nan', '2.0', '3.0', '5.0', '7.0',
       '6.0', '4.0', '0.0', '10.0', '1.0', '8.0', '12.0', 'Inactive']

In [404]:
dropped_cleaned = dropped[~dropped['propertyType'].isin(my_conditions)]
dropped_cleaned = dropped_cleaned[dropped_cleaned["lotSize"] != "Inactive"]
dropped_cleaned = dropped_cleaned[dropped_cleaned["squareFootage"] != "Inactive"]
dropped_cleaned = dropped_cleaned[dropped_cleaned["bedrooms"] != "Inactive"]
dropped_cleaned = dropped_cleaned[dropped_cleaned["bathrooms"] != "Inactive"]
dropped_cleaned = dropped_cleaned[dropped_cleaned["lotSize"].str.len().between(0, 8)]
dropped_cleaned = dropped_cleaned.dropna()
dropped_cleaned

,zipCode,propertyType,bedrooms,bathrooms,squareFootage,lotSize,yearBuilt,price,daysOnMarket
0,84037,Single Family,2.0,1.0,862.0,4792.0,1941.0,360000,92.0
1,84403,Single Family,3.0,1.0,1750.0,4792.0,1941.0,314999,92.0
4,84721,Single Family,5.0,4.0,3000.0,21780.0,2021.0,540000,85.0
5,84045,Single Family,3.0,2.5,4448.0,11326.0,2024.0,777400,83.0
6,84660,Single Family,3.0,2.0,1050.0,4792.0,1998.0,379900,82.0
...,...,...,...,...,...,...,...,...,...
82530,84319,Single Family,4.0,3.0,2646.0,25265.0,2020.0,650000,32.0
82531,84078,Single Family,3.0,2.0,2028.0,19166.0,2008.0,349000,44.0
82532,84078,Single Family,4.0,2.0,1620.0,10454.0,2008.0,240000,554.0
82534,84021,Single Family,7.0,3.0,3194.0,16988.0,2010.0,539999,8.0


In [405]:
dropped_cleaned.keys()

Index(['zipCode', 'propertyType', 'bedrooms', 'bathrooms', 'squareFootage',
       'lotSize', 'yearBuilt', 'price', 'daysOnMarket'],
      dtype='object')

In [406]:
#Bedrooms more than 10
#Bathrooms more than 8 and half
#Square feet more than 20000
#year built greater than 2024 get rid of it
#price under 15K



In [407]:
dropped_cleaned[['zipCode','bedrooms', 'bathrooms', 'squareFootage',
       'lotSize', 'yearBuilt', 'price', 'daysOnMarket']] = dropped_cleaned[['zipCode','bedrooms', 'bathrooms', 'squareFootage',
       'lotSize', 'yearBuilt', 'price', 'daysOnMarket']].apply(pd.to_numeric, errors="coerce")
dropped_cleaned = dropped_cleaned[dropped_cleaned['zipCode'] < 99998]
dropped_cleaned = dropped_cleaned[dropped_cleaned['bathrooms'] < 8.5]
dropped_cleaned = dropped_cleaned[dropped_cleaned['bedrooms'] < 10]
dropped_cleaned = dropped_cleaned[dropped_cleaned['squareFootage'] < 20000]
dropped_cleaned = dropped_cleaned[dropped_cleaned['yearBuilt'] > 1920]
dropped_cleaned = dropped_cleaned[dropped_cleaned['yearBuilt'] < 2025]
dropped_cleaned = dropped_cleaned[dropped_cleaned['bedrooms'] > 1]


In [408]:
le = LabelEncoder()
dropped_cleaned['propertyType'] = le.fit_transform(dropped_cleaned['propertyType'])
dropped_cleaned

,zipCode,propertyType,bedrooms,bathrooms,squareFootage,lotSize,yearBuilt,price,daysOnMarket
0,84037,1,2.0,1.0,862.0,4792.0,1941.0,360000.0,92.0
1,84403,1,3.0,1.0,1750.0,4792.0,1941.0,314999.0,92.0
4,84721,1,5.0,4.0,3000.0,21780.0,2021.0,540000.0,85.0
5,84045,1,3.0,2.5,4448.0,11326.0,2024.0,777400.0,83.0
6,84660,1,3.0,2.0,1050.0,4792.0,1998.0,379900.0,82.0
...,...,...,...,...,...,...,...,...,...
82530,84319,1,4.0,3.0,2646.0,25265.0,2020.0,650000.0,32.0
82531,84078,1,3.0,2.0,2028.0,19166.0,2008.0,349000.0,44.0
82532,84078,1,4.0,2.0,1620.0,10454.0,2008.0,240000.0,554.0
82534,84021,1,7.0,3.0,3194.0,16988.0,2010.0,539999.0,8.0


In [409]:
dropped_cleaned.describe()

zipCode            int64
propertyType       int32
bedrooms         float64
bathrooms        float64
squareFootage    float64
lotSize          float64
yearBuilt        float64
price            float64
daysOnMarket     float64
dtype: object

In [410]:
dropped_cleaned.dtypes

84790
2
9.0
8.0
19825.0
997960.0
2024.0
35000000.0
1092.0


In [ ]:
for x in dropped_cleaned:
    score = dropped_cleaned[x].max()
    print(score)


,zipCode,propertyType,bedrooms,bathrooms,squareFootage,lotSize,yearBuilt,price,daysOnMarket
count,52204.000000,52204.000000,52204.00000,52204.000000,52204.000000,52204.000000,52204.000000,5.220400e+04,52204.000000
mean,84257.564554,1.117654,3.87365,2.331335,2614.001705,14529.957704,1997.134147,7.095412e+05,69.440005
std,279.046234,0.461646,1.17936,0.971343,1299.672282,42530.079790,24.501796,8.408502e+05,80.508939
min,84001.000000,0.000000,2.00000,1.000000,400.000000,436.000000,1921.000000,9.750000e+03,1.000000
25%,84045.000000,1.000000,3.00000,2.000000,1717.000000,3485.000000,1980.000000,4.195000e+05,17.000000
50%,84098.000000,1.000000,4.00000,2.500000,2310.000000,8276.000000,2003.000000,5.250000e+05,45.500000
75%,84414.000000,1.000000,5.00000,3.000000,3226.000000,12197.000000,2019.000000,7.100000e+05,91.000000
max,84790.000000,2.000000,9.00000,8.000000,19825.000000,997960.000000,2024.000000,3.500000e+07,1092.000000


In [412]:
df = dropped_cleaned
for col in df.columns:
    try:
        pd.to_numeric(df[col])
    except ValueError:
        print(f"Non-numeric data found in column: {col}")

In [413]:
X = df.drop('price', axis=1)
y = df['price'] / 1e6

In [414]:
#Lasso Model

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
lasso = Lasso(alpha=0.01)
lasso_model = lasso.fit(X_train, y_train)

lass_prediction = lasso_model.predict(X_test)


In [415]:
lass_mse = mean_squared_error(y_test, lass_prediction)
lass_r2 = r2_score(y_test, lass_prediction)

print(lass_mse)
print(lass_r2)

0.4948536064352966
0.38056276126836386


In [416]:
#Linear Model
linear = LinearRegression()
linear.fit(X_train, y_train)


LinearRegression()

In [417]:
linear_prediction = linear.predict(X_test)

In [418]:
linear_mse = mean_squared_error(y_test, linear_prediction)
linear_r2 = r2_score(y_test, linear_prediction)

print(linear_mse)
print(linear_r2)

0.49498966077444106
0.3803924540844591


In [419]:
#SVC Model
svc = SVC(kernel="rbf", C=1.0, gamma="scale")
svc.fit(X_train, y_train)

ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

In [ ]:
svc_prediction = svc.predict(X_test)

In [ ]:
print(f"Accuracy: {accuracy_score(y_test, svc_prediction)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_test, svc_prediction)}")
print(f"Classification Report:\n{classification_report(y_test, svc_prediction)}")